In [1]:
import pandas as pd
import numpy as np
import re
import os
import nltk
import pymorphy2
import json

from os.path import join
from sklearn.model_selection import StratifiedShuffleSplit
from fasttext import load_model
from tqdm import tqdm

#visualization
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rc

In [2]:
def read_dataset(filepath, duplicates=False, clean=True):
    
    file = open(filepath, 'r', encoding='ISO-8859-1')
    data = pd.read_csv(file)
    
    old_names = data.keys()
    names = [n.encode('ISO-8859-1').decode('cp1251').encode('utf8') for n in old_names]
    names = [n.decode('utf-8') for n in names]
    
    new_data = dict()
    for old, new in zip(old_names, names):
        new_data[new] = list()
        for c in data[old]:
            try:
                s = c.encode('ISO-8859-1').decode('cp1251').encode('utf8')
                s = s.decode('utf-8')
                new_data[new].append(s)
            except AttributeError:
                new_data[new].append(c)
    
    new_data = pd.DataFrame(new_data, columns=['Описание', 'Категория жалобы'])
    new_data.rename(columns={'Описание': 'req', 'Категория жалобы': 'cat'}, inplace=True)
    new_data = new_data.dropna()  # dell nan
    if not duplicates:
        new_data = new_data.drop_duplicates()  # dell duplicates
    
    # как отдельную ветвь можно использовать
    if clean:
        delete_bad_symbols = lambda x: " ".join(re.sub('[^а-яa-zё0-9]', ' ', x.lower()).split())
        new_data['req'] = new_data['req'].apply(delete_bad_symbols)
    
    new_data = new_data.reset_index()
    
    return new_data

In [3]:
# загрузка csv файла из датасета
dataset_dir = './data/'
dataset_names = os.listdir(dataset_dir)
print('Avaliable datasets: {}'.format(dataset_names))

path = join(dataset_dir, dataset_names[0], 'vkusvill_all_categories.csv')
data = read_dataset(path, False)
print(data.keys())

Avaliable datasets: ['vkusvill']


/home/mks/intent/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (6,7,10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Index(['index', 'req', 'cat'], dtype='object')


In [4]:
## Разделение данных

In [ ]:
def train_test_split(data, part, req='req', cat='cat', names='train test', adres='./'):
    
    clas_names = data[cat].unique()
    num_cl = len(clas_names)
    data_mass = len(data[req])
    
    alpha = dict()
    for x in clas_names:
        alpha[x] = data[]
    